In [44]:
library(rJava)
library(xlsx)
library(Rsolnp)

In [45]:
product_types = read.xlsx("B8816-TakeHomeFinal-Fall2016.xlsx", sheetName = "question3-attributes")
data_prob3 = read.xlsx("B8816-TakeHomeFinal-Fall2016.xlsx", sheetName = "question3-wtp")

In [46]:
# Consolidating data from two sheets
data_combined = matrix(NA, nrow = nrow(data_prob3)*nrow(product_types), ncol = 8)
index = 0
for (i in 1:nrow(data_prob3)) {
  for (j in 1:nrow(product_types)) {
    index = index + 1
    data_combined[index,] = unlist(c(data_prob3[i,1:2], data_prob3[i,j+2], product_types[j,1:5]))
  }
}
data_combined = as.data.frame(data_combined)
colnames(data_combined) = c(names((data_prob3)[1:2]), "WTP", names(product_types))

In [47]:
# Linear model for traveler WTP
data_traveler = subset(data_combined, Type == 1)
traveler_WTP_model = lm(WTP ~ Screen + Weight + RAM + CPU, data = data_traveler)
summary(traveler_WTP_model)

# Linear model for general user WTP
data_general = subset(data_combined, Type == 2)
general_WTP_model = lm(WTP ~ Screen + Weight + RAM + CPU, data = data_general)
summary(general_WTP_model)

# Linear model for power user WTP
data_power = subset(data_combined, Type == 3)
power_WTP_model = lm(WTP ~ Screen + Weight + RAM + CPU, data = data_power)
summary(power_WTP_model)


Call:
lm(formula = WTP ~ Screen + Weight + RAM + CPU, data = data_traveler)

Residuals:
    Min      1Q  Median      3Q     Max 
-423.93 -104.34  -14.58   82.09  822.57 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1871.2770    17.0002  110.07   <2e-16 ***
Screen        32.6075     1.4612   22.32   <2e-16 ***
Weight      -205.8568     1.6123 -127.68   <2e-16 ***
RAM            8.2826     0.4916   16.85   <2e-16 ***
CPU           42.1295     3.0787   13.68   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 143 on 7195 degrees of freedom
Multiple R-squared:  0.7228,	Adjusted R-squared:  0.7227 
F-statistic:  4691 on 4 and 7195 DF,  p-value: < 2.2e-16



Call:
lm(formula = WTP ~ Screen + Weight + RAM + CPU, data = data_general)

Residuals:
    Min      1Q  Median      3Q     Max 
-724.65 -199.19  -31.07  148.93 1575.35 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  991.357     25.452   38.95   <2e-16 ***
Screen        56.286      2.188   25.73   <2e-16 ***
Weight      -103.968      2.414  -43.07   <2e-16 ***
RAM           18.985      0.736   25.80   <2e-16 ***
CPU           96.981      4.609   21.04   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 276.4 on 11995 degrees of freedom
Multiple R-squared:  0.2409,	Adjusted R-squared:  0.2406 
F-statistic: 951.5 on 4 and 11995 DF,  p-value: < 2.2e-16



Call:
lm(formula = WTP ~ Screen + Weight + RAM + CPU, data = data_power)

Residuals:
    Min      1Q  Median      3Q     Max 
-176.45  -57.75  -13.42   36.58  439.94 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -774.3073    10.4690  -73.96   <2e-16 ***
Screen       100.8356     0.8999  112.06   <2e-16 ***
Weight       -21.1604     0.9929  -21.31   <2e-16 ***
RAM           30.1614     0.3027   99.63   <2e-16 ***
CPU          197.3537     1.8959  104.10   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 71.91 on 4795 degrees of freedom
Multiple R-squared:  0.9586,	Adjusted R-squared:  0.9586 
F-statistic: 2.779e+04 on 4 and 4795 DF,  p-value: < 2.2e-16


In [48]:
# Estimation of MNL model for travelers
data_traveler = subset(data_prob3, Type == 1)
var_traveler = apply(data_traveler[,3:26], 2, var)
mu_traveler = sqrt(mean(var_traveler) * 6) / pi

# MNL model shape parameter for travelers
round(mu_traveler, digits=2)

# MNL model for travelers incorporating linear model for WTP based on design attributes
traveler_demand_MNL = function(inputs){
  
  # each product represents a column in the inputs matrix
  attributes = inputs[1:4,]
  prices = inputs[5,]
  
  # Estimating u coefficients from linear model for WTP
  u = rep(NA,ncol(attributes))
  for (i in 1:ncol(attributes)) {
    product_parameters = c(1,attributes[,i])
    u[i] = sum(traveler_WTP_model$coefficients * product_parameters)
  }
  
  # Estimating choice probabilities
  attract = exp((u - prices) / mu_traveler)
  choice = attract / (sum(attract) + 1)
  return(choice)
}

[1] 107.93

In [49]:
# Estimation of MNL model for general users
data_general = subset(data_prob3, Type == 2)
var_general = apply(data_general[,3:26], 2, var)
mu_general = sqrt(mean(var_general) * 6) / pi

# MNL model shape parameter for general users
round(mu_general, digits=2)

# MNL model for general users incorporating linear model for WTP based on design attributes
general_demand_MNL = function(inputs){
  
  # each product represents a column in the inputs matrix
  attributes = inputs[1:4,]
  prices = inputs[5,]
  
  # Estimating u coefficients from linear model for WTP
  u = rep(NA,ncol(attributes))
  for (i in 1:ncol(attributes)) {
    product_parameters = c(1,attributes[,i])
    u[i] = sum(general_WTP_model$coefficients * product_parameters)
  }
  
  # Estimating choice probabilities
  attract = exp((u - prices) / mu_general)
  choice = attract / (sum(attract) + 1)
  return(choice)
}

[1] 214.93

In [50]:
# Estimation of MNL model for power users
data_power = subset(data_prob3, Type == 3)
var_power = apply(data_power[,3:26], 2, var)
mu_power = sqrt(mean(var_power) * 6) / pi

# MNL model shape parameter for power users
round(mu_power, digits=2)

# MNL model for power users incorporating linear model for WTP based on design attributes
power_demand_MNL = function(inputs){
  
  # each product represents a column in the inputs matrix
  attributes = inputs[1:4,]
  prices = inputs[5,]
  
  # Estimating u coefficients from linear model for WTP
  u = rep(NA,ncol(attributes))
  for (i in 1:ncol(attributes)) {
    product_parameters = c(1,attributes[,i])
    u[i] = sum(power_WTP_model$coefficients * product_parameters)
  }
  
  # Estimating choice probabilities
  attract = exp((u - prices) / mu_power)
  choice = attract / (sum(attract) + 1)
  return(choice)
}

[1] 55.93

In [51]:
# Inputting example data
Example_attributes = t(rbind(product_types[1,2:5],product_types[10,2:5],product_types[24,2:5]))
Example_prices = c(1400, 1600, 1700)
Example_matrix = rbind(Example_attributes, Example_prices)
Example = as.data.frame(Example_matrix)

In [52]:
# Estimating choices for each segment
Example_result_traveler = 300*traveler_demand_MNL(Example)
Example_result_traveler[4] = 300 - sum(Example_result_traveler[1:3])
Example_result_general = 500*general_demand_MNL(Example)
Example_result_general[4] = 500 - sum(Example_result_general[1:3])
Example_result_power = 200*power_demand_MNL(Example)
Example_result_power[4] = 200 - sum(Example_result_power[1:3])

In [53]:
# Formatting output table with choices
Example_result = rbind(Example_result_traveler, Example_result_general, Example_result_power)
colnames(Example_result) = c(paste("Product",colnames(Example)),"No buy")
rownames(Example_result) = c("traveler", "general", "power")

# Choices of each of the segments
round(Example_result, 0)

,Product 1,Product 10,Product 24,No buy
traveler,288,3,0,9
general,203,81,147,69
power,0,0,198,2


In [54]:
# Cost per product function
Product_cost = function(product_attributes){
  cost_factors = c(45,-70,5,100)
  product_atts = as.vector(product_attributes)
  cost = 250 + sum(cost_factors*product_atts)
  return(cost)
}

In [55]:
# Total profit function for a vector of 2 products
Profit3 = function(inputs){
  
  # Parsing input data
  inputs = matrix(inputs,nrow = 5,ncol = 2)
  attributes = inputs[1:4,]
  prices = inputs[5,]

  # Calculating demand per segment for both products
  demand_traveler = 300*traveler_demand_MNL(rbind(attributes,prices))
  demand_general = 500*general_demand_MNL(rbind(attributes,prices))
  demand_power = 200*power_demand_MNL(rbind(attributes,prices))
  
  # Calculating unit cost for both products
  costs = c(Product_cost(attributes[,1]),Product_cost(attributes[,2]))

  # Calculating profit per segment  
  profit_traveler = sum(demand_traveler*(prices-costs))
  profit_general = sum(demand_general*(prices-costs))
  profit_power = sum(demand_power*(prices-costs))
  
  # Calculating total profit across all segments
  total_profit = profit_traveler + profit_general + profit_power
  return(-total_profit)
}

In [56]:
# Solving optimization problem
x1_attributes = c(13,3,4,1.5)
x1_price = 500
x2_attributes = c(17,7,16,3.5)
x2_price = 500
x_matrix = cbind(c(x1_attributes, x1_price), c(x2_attributes, x2_price))
result_profit = gosolnp(pars=as.vector(x_matrix), fun=Profit3, LB = c(13,3,4,1.5,500,13,3,4,1.5,500), UB = c(17,7,16,3.5,5000,17,7,16,3.5,5000), n.restarts = 10, n.sim = 100000)


Iter: 1 fn: -700301.9175	 Pars:    17.00000    7.00000   16.00000    3.50000 1817.31946   13.00000    3.00000   16.00000    1.50000 1687.60330
Iter: 2 fn: -700301.9188	 Pars:    17.00000    7.00000   16.00000    3.50000 1817.31941   13.00000    3.00000   16.00000    1.50000 1687.60356
solnp--> Completed in 2 iterations

Iter: 1 fn: -700301.9137	 Pars:    17.00000    7.00000   16.00000    3.50000 1817.31945   13.00000    3.00000   16.00000    1.50000 1687.60362
Iter: 2 fn: -700301.9161	 Pars:    17.00000    7.00000   16.00000    3.50000 1817.31929   13.00000    3.00000   16.00000    1.50000 1687.60396
solnp--> Completed in 2 iterations

Iter: 1 fn: -700301.9174	 Pars:    13.00000    3.00000   16.00000    1.50000 1687.60384   17.00000    7.00000   16.00000    3.50000 1817.31924
Iter: 2 fn: -700301.9188	 Pars:    13.00000    3.00000   16.00000    1.50000 1687.60364   17.00000    7.00000   16.00000    3.50000 1817.31922
solnp--> Completed in 2 iterations

Iter: 1 fn: -700301.9145	 Pars:  

In [57]:
# Reporting optimization results
result_pars = matrix(result_profit$pars,nrow = 5,ncol = 2)
colnames(result_pars) = c("Product 1", "Product 2")
rownames(result_pars) = c(names(product_types[,2:5]),"Price")
round(result_pars,1)

,Product 1,Product 2
Screen,17.0,13.0
Weight,7.0,3.0
RAM,16.0,16.0
CPU,3.5,1.5
Price,1817.3,1687.6


In [58]:
# Expected profit at optimal design and pricing
-Profit3(as.vector(result_pars))

[1] 700301.9